In [ ]:
from google.colab import drive
drive.mount('/content/drive')



In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

sns.set(rc={'figure.figsize': (25, 10)}, font_scale=1.5)

# df = pd.read_csv('/content/drive/MyDrive/ECH_2019.csv')

df = pd.read_csv('ECH_2019.csv')


In [2]:
df.sample(3)


,id_hogar,sexo,edad,ascendencia,parentesco,estado_laboral,sueldo,busca_otro_trabajo,actividad,asistencia alguna vez a enseñanza,edu preescolar,edu primaria,edu media,edu técnica,edu univ o simil,edu terciario no uni,edu posgrado,edu mag prof
87708,2019047215,Mujer,75,Blanca,Jefe/a de hogar,Inactivo: jubilado,0,0,.,Sí,No asistió,"Sí, asistió",0,No asistió,0,0,0,0
76771,2019041449,Mujer,40,Blanca,Jefe/a de hogar,Inactivo: realiza los quehaceres del hogar,0,0,.,Sí,"Sí, asistió","Sí, asistió","Sí, asistió",No asistió,0,0,0,0
95827,2019051583,Mujer,68,Blanca,Esposo/a o compañero/a,Inactivo: jubilado,0,0,.,Sí,No asistió,"Sí, asistió","Sí, asistió",No asistió,"Sí, asistió",No asistió,"Sí, asistió",No asistió
8729,2019004669,Mujer,30,Blanca,Esposo/a o compañero/a,Inactivo: realiza los quehaceres del hogar,0,0,.,Sí,"Sí, asistió","Sí, asistió",No asistió,No asistió,0,0,0,0
49406,2019026549,Mujer,60,Blanca,Jefe/a de hogar,Inactivo: realiza los quehaceres del hogar,0,0,.,Sí,"Sí, asistió","Sí, asistió","Sí, asistió",No asistió,No asistió,No asistió,0,No asistió


In [3]:
no_data = {0: 'No hay dato', '0': 'No hay dato'}

mask = df.columns.difference(['edad', 'sueldo', 'hijos'])

df[mask] = df[mask].replace(no_data)

df.sample(3)



,id_hogar,sexo,edad,ascendencia,parentesco,estado_laboral,sueldo,busca_otro_trabajo,actividad,asistencia alguna vez a enseñanza,edu preescolar,edu primaria,edu media,edu técnica,edu univ o simil,edu terciario no uni,edu posgrado,edu mag prof
25464,2019013748,Mujer,0,Blanca,Hijo/a solo del jefe/a,Menores de 14 años,0,No hay dato,.,No,No hay dato,No hay dato,No hay dato,No hay dato,No hay dato,No hay dato,No hay dato,No hay dato
8335,2019004447,Mujer,74,Blanca,Jefe/a de hogar,Inactivo: jubilado,0,No hay dato,.,Sí,No asistió,"Sí, asistió","Sí, asistió",No asistió,"Sí, asistió",No asistió,No hay dato,No asistió
97617,2019052554,Mujer,48,Blanca,Jefe/a de hogar,Ocupados,70000,No,Enseñanza superior,Sí,"Sí, asistió","Sí, asistió","Sí, asistió",No asistió,"Sí, asistió",No asistió,Asiste actualmente,No asistió
51754,2019027770,Mujer,18,Blanca,Hijo/a de ambos,Inactivo: estudiante,0,No hay dato,.,Sí,"Sí, asistió","Sí, asistió",Asiste actualmente,No asistió,No hay dato,No hay dato,No hay dato,No hay dato
67484,2019036330,Hombre,67,Blanca,Jefe/a de hogar,Ocupados,16000,No,Elaboración de alimentos preparados para animales,Sí,No asistió,"Sí, asistió",No asistió,No asistió,No hay dato,No hay dato,No hay dato,No hay dato


Seleccionamos personas con edad suficiente para trabajar

In [ ]:
# df = df[~df['estado_laboral'].str.contains('Menores de 14 años')]



Creamos una columna que cuenta la cantidad de hijos por grupo familiar

In [4]:
df['hijos'] = (
    df.groupby('id_hogar')['id_hogar'].transform('count') - 1
)



# CATEGORIZAR NUMERICOS

In [5]:
df['rango_sueldos'] = pd.cut(df['sueldo'], bins=[
    0,
    5000,
    10000,
    30000,
    50000,
    100000,
    float('Inf'),
    ]).astype('category')

df['rango_edades'] = pd.cut(df['edad'], bins=[
    0,
    10,
    15,
    20,
    30,
    40,
    50,
    60,
    70,
    float('Inf'),
    ]).astype('category')

# labels=['0-5000', 'yyy', 'zzz']



### AJUSTANDO DATOS PARA ENTRENAR EL MODELO

Primero encodificamos las variables categóricas

Eliminanos las columnas que no vamos a usar o que ya categorizamos

In [12]:
df = df[[
    'sexo',
    'ascendencia',
    'estado_laboral',
    'busca_otro_trabajo',
    'actividad',
    'asistencia alguna vez a enseñanza',
    'edu preescolar',
    'edu primaria',
    'edu media',
    'edu técnica',
    'edu univ o simil',
    'edu terciario no uni',
    'edu posgrado',
    'edu mag prof',
    'parentesco',
    'hijos',
    'rango_sueldos',
    'rango_edades',
    ]]

In [18]:
df['estado_laboral'].unique()

array(['Inactivo: jubilado', 'Inactivo: realiza los quehaceres del hogar',
       'Ocupados', 'Menores de 14 años', 'Inactivo: estudiante',
       'Desocupados propiamente dichos', 'Inactivo: pensionista',
       'Inactivo: otro', 'Desocupados en seguro de paro',
       'Desocupados buscan trabajo por primera vez', 'Inactivo: rentista'],
      dtype=object)

## Reducimos la consideración de estado laboral a menos categorías

In [16]:
def change_cat(value):
    if 'Desocupados' in value:
        return 'Desocupado'
    if 'Inactivo' in value:
        return 'Inactivo'
    if 'Menores' in value:
        return 'Menor'
    else:
        return 'Ocupado'

df['estado_laboral'] = df['estado_laboral'].apply(change_cat, inplace=True)

df['estado_laboral'].unique()

array(['Inactivo', 'Ocupado', 'Menor', 'Desocupado'], dtype=object)

### DEFINIMOS LAS VARIABLES DEPENDIENTES E INDEPENDIENTES

In [ ]:
# Feautures
X = df.drop('estado_laboral', axis=1).to_numpy()
y = df['estado_laboral']

print('X shape', X.shape, '\nY shape', y.shape)



In [ ]:
np.unique(y)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot = OneHotEncoder()

X = one_hot.fit_transform(X)



### Dividimos los datos en un 30% de test y 70% de entrenamiento

In [ ]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42) 



In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)



# DEFINIMOS ALGORITMO DE REGRESIÓN LOGISTICA

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

logistic_reg = LogisticRegression(max_iter=20000)

logistic_reg.fit(X_train, y_train)

predictions = logistic_reg.predict(X_test)



### VALOR DE CROSS VALIDATION PARA CADA FOLD

In [ ]:
scores = cross_val_score(logistic_reg, X, y, cv=10)

scores



In [ ]:
print('Promedio de cross_validation ', round(scores.mean(),2))
print('Score en test ', round(logistic_reg.score(X_test, y_test),2))
print('Score en train ', round(logistic_reg.score(X_train, y_train),2))



In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, predictions))



### INVESTIGAMOS LA EFECTIVIDAD DEL MODELO SOBRE LOS DATOS

In [ ]:
from sklearn.metrics import accuracy_score

#Prediccion en Train
y_train_pred = logistic_reg.predict(X_train)

#Prediccion en Test
y_test_pred = logistic_reg.predict(X_test)

#Calculo el accuracy en Train
train_accuracy = accuracy_score(y_train, y_train_pred)

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('% de aciertos sobre el set de entrenamiento:', train_accuracy)
print('% de aciertos sobre el set de evaluación:',test_accuracy)



### DEFINIMOS UNA PREDICCIÓN Y LA APLICAMOS SOBRE EL MODELO

In [ ]:
prediction = X.toarray()

prediction = prediction[1000].reshape(1, -1)
print(logistic_reg.predict(prediction))
print(y_train[1000])



Determinamos la probabilidad de la predicción

In [ ]:
probabilidades_prediccion = logistic_reg.predict_proba(prediction)
probabilidades_prediccion[:,1]



# REPETIMOS EL EJERCICIO CON EL MODELO DE DECISSION TREE

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(100)

random_forest.fit(X_train, y_train)

forest_prediction = random_forest.predict(X_test)



In [ ]:
scores = cross_val_score(random_forest, X, y, cv=10)

scores



In [ ]:
print('Promedio de cross_validation ', round(scores.mean(),2))
print('Score en test ', round(logistic_reg.score(X_test, y_test),2))
print('Score en train ', round(logistic_reg.score(X_train, y_train),2))



In [ ]:
#Prediccion en Train
y_train_pred = random_forest.predict(X_train)

#Prediccion en Test
y_test_pred = random_forest.predict(X_test)

#Calculo el accuracy en Train
train_accuracy = accuracy_score(y_train, y_train_pred)

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('% de aciertos sobre el set de entrenamiento:', train_accuracy)
print('% de aciertos sobre el set de evaluación:',test_accuracy)



In [ ]:
print(random_forest.predict(prediction))
print(y_train[1000])



# REPETIMOS EL EJERCICIO CON EL MODELO DE SUPPORT VECTOR MACHINE

In [ ]:
from sklearn import svm

svm_model = svm.SVC(kernel='linear')

svm_model.fit(X_train, y_train)

svm_model.predict(X_test)



In [ ]:
scores = cross_val_score(svm_model, X, y, cv=10)

scores



In [ ]:
print('Promedio de cross_validation ', round(scores.mean(),2))
print('Score en test ', round(svm_model.score(X_test, y_test),2))
print('Score en train ', round(svm_model.score(X_train, y_train),2))



In [ ]:
#Prediccion en Train
y_train_pred = svm_model.predict(X_train)

#Prediccion en Test
y_test_pred = svm_model.predict(X_test)

#Calculo el accuracy en Train
train_accuracy = accuracy_score(y_train, y_train_pred)

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('% de aciertos sobre el set de entrenamiento:', train_accuracy)
print('% de aciertos sobre el set de evaluación:',test_accuracy)



In [ ]:
print(svm_model.predict(prediction))
print(y_train[1000])

